# Final Project
## ADSP 32021 IP01 Machine Learning Operations
### 2: Document Pre-Processing
#### Group 2: Maria Clarissa Fionalita, Kajal Shukla, Mia Zhang, Priya Suvvaru Venkata

In [25]:
!python --version
!jupyter nbextension enable --py widgetsnbextension

Python 3.10.13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [26]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "cpu"

In [27]:
from langchain.docstore.document import Document
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import json
from pathlib import Path
from pprint import pprint
import ast

In [28]:
!git log --oneline

1b19e4a (HEAD -> master) reupload RAG data v2 pkl: *_consec.json
6cafe87 reupload RAG data v2 index: *_.json
79c43fc RAG data v2 pkl: *_consec.json
d39b552 RAG data v2 index: *_.json
68af42d RAG data v1 pkl: *_consec.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03af77e RAG data v1: *_consec.json
059a4ce full data v2: *.json
b8b53f2 full data v1: *_consec.json
7a85b8c test data v2: combine v1 with pre-processed test_webmd_squad_v2_full.json
9868399 test data v1: pre-processed test_webmd_squad_v2_consec.json
bb55d3c validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


# Load Text Embedding Model

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [30]:
# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-l6-v2",     # Provide the pre-trained model's path
    model_kwargs = model_kwargs, # Pass the model configuration options
    encode_kwargs = encode_kwargs # Pass the encoding options
)

In [31]:
# testing the embedding

query_result = embeddings.embed_query("This is a test document.")
query_result[:3]

[-0.03833853453397751, 0.12346465140581131, -0.02864295244216919]

# V1 Document
["Question Answering"](https://huggingface.co/docs/transformers/tasks/question_answering)

Only contains document from ```*_consec.json```

In [32]:
!git checkout b8b53f2 data/full_data.json.dvc
!dvc checkout

Updated 1 path from e0e006f


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Building workspace index                              |14.0 [00:00,  173entry/s]
Comparing indexes                                    |15.0 [00:00, 3.65kentry/s]
ERROR: Can't remove the following unsaved files without confirmation. Use `--force` to force.
/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operations/Final Project/data/RAG_data/index.faiss
/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operations/Final Project/data/RAG_data/index.pkl


In [33]:
file_path = "data/full_data.json"

## Load Data

In [34]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["url"] = record.get('url')
    metadata["id"] = record.get('id')
    metadata["question"] = record.get('question')
    metadata["context"] = record.get('context')

    return metadata

In [35]:
loader = JSONLoader(
    file_path = file_path,
    jq_schema = ".[]",
    content_key = "answers",
    metadata_func= metadata_func,
    text_content = False
)

data = loader.load()

## Clean Document

In [36]:
for i in range(len(data)):
    pg_content = ast.literal_eval(data[i].page_content)[0]
    text = pg_content["text"]
    len_text = len(text)
    
    data[i].page_content = text
    data[i].metadata["answer_span"] = pg_content["answer_span"]
    data[i].metadata["answer_start"] = pg_content["answer_start"]
    data[i].metadata["answer_starts"] = pg_content["answer_starts"]


KeyboardInterrupt



In [ ]:
len(data)

In [ ]:
pprint(data[10])

## Document Transformers

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 150,
    length_function = len,
    is_separator_regex = False,
)

## Create Vector Stores

[Saving and Loading](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading)

In [ ]:
data_size = len(data)
print(data_size)

In [ ]:
%%time
db = None

# for i in range(0, 4, 2):
for i in range(0, data_size, 2):
    if i % 10 == 0:
        print(i)
    db_1 = FAISS.from_documents(data[i : i + 2], embeddings)

    if db == None:
        db = db_1
    elif db != None:
        db.merge_from(db_1)

    db.save_local("data/RAG_data")

### Push Full Data V1

In [ ]:
# upload version 1 of the full data
!dvc add data/RAG_data/index.faiss
!git add data/.gitignore data/RAG_data/index.faiss.dvc

# push the v1 of the full data
!git commit -m "RAG data v1: *_consec.json"
!cat data/RAG_data/index.faiss.dvc
!dvc push

In [ ]:
!dvc add data/RAG_data/index.pkl
!git add data/.gitignore data/RAG_data/index.pkl.dvc

# push the v1 of the full data
!git commit -m "RAG data v1 pkl: *_consec.json"
!cat data/RAG_data/index.pkl.dvc
!dvc push

In [ ]:
!git log --oneline

# V2 Document

Contains all ```*.json```

In [37]:
# checkout full_data.json v2

!git checkout 059a4ce data/full_data.json.dvc
!dvc checkout -f

Updated 1 path from 1b84350


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Building workspace index                              |14.0 [00:00,  234entry/s]
Comparing indexes                                    |15.0 [00:00, 3.57kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Ma0.00/358M [00:00<?,        ?B/s]
  9%|▉         |/mnt/d/UChicago/Q4/ADSP 32034.0M/358M [00:00<00:00,     349MB/s]
 20%|██        |/mnt/d/UChicago/Q4/ADSP 32072.0M/358M [00:00<00:00,     377MB/s]
 30%|███       |/mnt/d/UChicago/Q4/ADSP 3202109M/358M [00:00<00:00,     380MB/s]
 41%|████      |/mnt/d/UChicago/Q4/ADSP 3202146M/358M [00:00<00:00,     380MB/s]
 52%|█████▏    |/mnt/d/UChicago/Q4/ADSP 3202185M/358M [00:00<00:00,     388MB/s]
 63%|██████▎   |/mnt/d/UChicago/Q4/ADSP 3202225M/358M [00:00<00:00,     398MB/s]
 74%|███████▍  |/mnt/d/UChicago/Q4/ADSP 3202265M/358M [00:00<00:00,     402MB/s]
 85%|████████▌ |/mnt/d/UCh

In [39]:
file_path = "data/full_data.json"

## Load Data

In [40]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["url"] = record.get('url')
    metadata["id"] = record.get('id')
    metadata["question"] = record.get('question')
    metadata["context"] = record.get('context')[0]

    return metadata

In [41]:
loader = JSONLoader(
    file_path = file_path,
    jq_schema = ".[]",
    content_key = "answers",
    metadata_func= metadata_func,
    text_content = False
)

data = loader.load()

In [42]:
data[0]

Document(page_content='[{\'text\': "If the glaucoma does not respond to medication, or if you cannot tolerate the side effects, your doctor may change medications or recommend one of several surgical techniques: Laser trabeculoplasty creates small laser burns in the area where the fluid drains, improving the outflow rate of aqueous fluid. This relatively brief procedure can often be done in an ophthalmologist\'s clinic. Trabeculectomy is a surgical procedure that creates a new channel for fluid outflow in cases in which the intraocular pressure is high and the optic nerve damage progresses. Long-term results vary, but generally, the success rate is good. Surgical implants that shunt fluid out of the eye may also be used to decrease pressure in the eye. Remember, all forms of medical or surgical treatment have potential benefits and risks. Before giving your consent, always ask the surgeon to clearly explain any treatment or surgery as well as the proposed benefits, effective alternativ

## Clean Document

In [43]:
for i in range(len(data)):
    pg_content = ast.literal_eval(data[i].page_content)[0]
    text = pg_content["text"]
    len_text = len(text)
    
    data[i].page_content = text

In [44]:
len(data)

60097

In [45]:
pprint(data[20])

Document(page_content='Some drugs raise your chance of having infections, so you should avoid picking up dog waste or cleaning litter boxes, bird cages, and fish tanks. Ask your doctor how you can stay safe around your pets, and wash your hands after touching any animals.', metadata={'source': '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operations/Final Project/data/full_data.json', 'seq_num': 21, 'url': 'https://www.webmd.com/cancer/qa/can-i-still-care-for-my-pet-during-chemotherapy', 'id': 'c2a7c00cdc9f4da477cfecee20cb25f5', 'question': 'Can I still care for my pet during chemotherapy?', 'context': "A chemotherapy session may take only a few hours, but you might have side effects for days or weeks afterward. To make your life easier and more comfortable as you recover, think about how you'll take care of yourself at home before you go in for treatment. 1. Ask someone to drive you to and from treatments. You might feel fine after a session, or you could feel tired and unsafe behin

## Document Transformers

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 150,
    length_function = len,
    is_separator_regex = False,
)

In [47]:
data_size = len(data)
print(data_size)

60097


In [52]:
%%time
db = None

# for i in range(0, 10, 2):
for i in range(0, data_size, 2):
    if i % 10 == 0:
        print(i)
    db_1 = FAISS.from_documents(data[i : i + 2], embeddings)

    if db == None:
        db = db_1
    elif db != None:
        db.merge_from(db_1)

    db.save_local("data/RAG_data")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

### Push Full Data V2

In [56]:
# upload v2 of the full data
!dvc add data/RAG_data/index.faiss
!git add data/.gitignore data/RAG_data/index.faiss.dvc

# push the v2 of the full db index
!git commit -m "fixed RAG data v2 index: *_.json"
!cat data/RAG_data/index.faiss.dvc
!dvc push

!dvc add data/RAG_data/index.pkl
!git add data/.gitignore data/RAG_data/index.pkl.dvc

# push the v2 of the full db pkl
!git commit -m "fixed RAG data v2 pkl: *_consec.json"
!cat data/RAG_data/index.pkl.dvc
!dvc push

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  1.51files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  1.14file/s]

To track the changes with git, run:

	git add data/RAG_data/index.faiss.dvc

To enable auto staging, run:

	dvc config core.autostage true


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[master 7eabf31] fixed RAG data v2 index: *_.json
 1 file changed, 1 deletion(-)
outs:
- md5: 10df7fe5f02904b269f92a6c262de588
  size: 92309037
  hash: md5
  path: index.faiss


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:01<00:00,  1.07s/files]
100% Adding...|████████████████████████████████████████|1/1 [00:01,  1.23s/file]

To track the changes with git, run:

	git add data/RAG_data/index.pkl.dvc

To enable auto staging, run:

	dvc config core.autostage true


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[master dfab4a5] fixed RAG data v2 pkl: *_consec.json
 1 file changed, 1 deletion(-)
outs:
- md5: 769939f134987738897bd5885c8399b2
  size: 367384660
  hash: md5
  path: index.pkl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Pushing                                               |0.00 [00:00,     ?file/s]
!
  0% Checking cache in '1wa6Ax2ro0D7EtdOYmmg0O7PmT8sfFGTN/files/md5'| |0/? [00:0
  0% Querying cache in '1wa6Ax2ro0D7EtdOYmmg0O7PmT8sfFGTN/files/md5'| |1/256 [00
Everything is up to date.                                                       


In [57]:
!git log --oneline

dfab4a5 (HEAD -> master) fixed RAG data v2 pkl: *_consec.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


7eabf31 fixed RAG data v2 index: *_.json
c74056d fixed RAG data v2 pkl: *_consec.json
413f40f fixed RAG data v2 index: *_.json
1b19e4a reupload RAG data v2 pkl: *_consec.json
6cafe87 reupload RAG data v2 index: *_.json
79c43fc RAG data v2 pkl: *_consec.json
d39b552 RAG data v2 index: *_.json
68af42d RAG data v1 pkl: *_consec.json
03af77e RAG data v1: *_consec.json
059a4ce full data v2: *.json
b8b53f2 full data v1: *_consec.json
7a85b8c test data v2: combine v1 with pre-processed test_webmd_squad_v2_full.json
9868399 test data v1: pre-processed test_webmd_squad_v2_consec.json
bb55d3c validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage
